In [ ]:


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'agricultural-pests-image-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3032186%2F5212747%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T091535Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6e4c62df8b57787d84df870b65907dc05b1431737292974dfef813a2f00e76550d4d806c26268f658b65fdf5ce72bdabf9c6fe84f74a76dd3305d604e3b06c10456e6fddc9d65f3ce8dca8cfe66bdefa2ada6d94787dfb38b6468b37f41ffa05075bcb2dd579d9ef47c054b28bc6482f6fd921f95812adf39c9e08b441e3e6486077d61ab80695a8b5980e2b03a2d8a2ce12adb3e0aa814bd74827b9f7bc231b5cae4e67c1a7699d2712cc31ce91d5273ce72cc62b6c567446b99e2e984f5655dd5c453f709001a18d1be6e12b5896b4715a92b140567f18237fa44eb41187d0b64b2b8f617273f3008cc14ae6d157e6bd158596cfa33b7e616b6b37e3a2b58e,agricultural-pests-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3087212%2F5311827%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T091535Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8195c842f5544653b42bcb7f25f4311a62caa3166334495570b9e9b8fabb3fd436c4c3f17b8d30539d919e0b6277574f1d72ed69b4fdaa46bd8cd00df1ebe48bbd384f48d06e57690d83721e9cae5a78578212f7a0663aa3df0ba37760819ddbabc7e7dbdcb74dac3eaae592f7b4189f474fe2bb0a24913762103900f48d60b95c6e192d091d7e3fc88b797db8b8e895e7e27a604c444ba54eb274438c0c3e23e74fb1bd5f705b32b60ea9b1a7a0bad2a92dfc492d67634c59f9faba1c576188e35f64f651a5b14bc773e1d764399efc8a2ca651afc4ec950323d91a2a684affa0cc1aa28b9041d0458baa15751e125475cbb175e1b8136266bb88fd846fd927,resnet-50-v2-agricultural-pests-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3088601%2F5314501%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T091535Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D59c0886e8b13c250ba651cba6ae701a9cf47aab8c285d7c633c4c102d9f4e71ad2218483c405ff96e91bc35250eaae7c4636a431e246a5015d32cff597bb8d10390593da7534ea8c02ceb24f2854db729113c359d41101e79606c77262674e6cd2ea8b5f733a77a8e211041be403456cf3afc6288de56060ec0846b7c9b29aea663c472908c2b8eace586bd26486dd605496cfead8a2f64edada4e151f5b608015e416db935a50e93a3eeafd2c77729f870f00b0e167dd4ded871be2d61a6b9d90a98a131fc8501f167ac1cca14b941fca10ad4d42299c3d2662744966f8aab634faa37d39d2aee4d84b8a61e1fee397132b2b58db49c03d94a819daf4c02e75'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 107330814 bytes downloaded
Downloaded and uncompressed: agricultural-pests-image-dataset
[==================================================] 107426319 bytes downloaded
Downloaded and uncompressed: agricultural-pests-dataset
[==================================================] 97615574 bytes downloaded
Downloaded and uncompressed: resnet-50-v2-agricultural-pests-classification
Data source import complete.


In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/agricultural-pests-dataset/train.csv')
train.head()

,filename,label
0,/kaggle/input/agricultural-pests-image-dataset...,snail
1,/kaggle/input/agricultural-pests-image-dataset...,wasp
2,/kaggle/input/agricultural-pests-image-dataset...,bees
3,/kaggle/input/agricultural-pests-image-dataset...,grasshopper
4,/kaggle/input/agricultural-pests-image-dataset...,weevil


In [ ]:
test=pd.read_csv('/kaggle/input/agricultural-pests-dataset/test.csv')
test.head()

,filename,label
0,/kaggle/input/agricultural-pests-image-dataset...,wasp
1,/kaggle/input/agricultural-pests-image-dataset...,snail
2,/kaggle/input/agricultural-pests-image-dataset...,catterpillar
3,/kaggle/input/agricultural-pests-image-dataset...,weevil
4,/kaggle/input/agricultural-pests-image-dataset...,beetle


In [ ]:
train['filename']=train['filename'].str.replace('/kaggle/input/agricultural-pests-image-dataset/','/kaggle/input/agricultural-pests-dataset/train/')
train.head()

,filename,label
0,/kaggle/input/agricultural-pests-dataset/train...,snail
1,/kaggle/input/agricultural-pests-dataset/train...,wasp
2,/kaggle/input/agricultural-pests-dataset/train...,bees
3,/kaggle/input/agricultural-pests-dataset/train...,grasshopper
4,/kaggle/input/agricultural-pests-dataset/train...,weevil


In [ ]:
import PIL
import numpy as np

# Assuming your model expects 224x224 images
IMAGE_SIZE = (224, 224)

def preprocess_image(image_path):
  # Load image using PIL
  img = Image.open(image_path)
  # Resize image
  img = img.resize(IMAGE_SIZE)
  # Convert to a NumPy array (optional, depending on model)
  img_array = np.array(img)
  # ... (further normalization if needed)
  return img_array

# ... (existing code for reading CSV files)

# Apply preprocessing to training and test image paths
train['image'] = train['filename'].apply(lambda x: preprocess_image(x))
test['image'] = test['filename'].apply(lambda x: preprocess_image(x))


In [ ]:
! pip install lobe[all]


In [ ]:
import lobe


In [ ]:
from lobe import ImageModel

model = ImageModel.load('/kaggle/input/resnet-50-v2-agricultural-pests-classification/train TensorFlow')
model

In [ ]:
result=[]
for i in test.filename:
    result_i= model.predict_from_file(i)
    result.append(result_i.prediction)
result[:5]

['wasp', 'snail', 'catterpillar', 'weevil', 'beetle']

In [ ]:
test=test.assign(prediction=result)
test.head()

,filename,label,prediction,image
0,/kaggle/input/agricultural-pests-dataset/test/...,wasp,wasp,"[[[163, 156, 150], [163, 156, 150], [164, 157,..."
1,/kaggle/input/agricultural-pests-dataset/test/...,snail,snail,"[[[44, 46, 41], [37, 39, 34], [38, 40, 35], [3..."
2,/kaggle/input/agricultural-pests-dataset/test/...,catterpillar,catterpillar,"[[[186, 188, 148], [188, 190, 150], [188, 190,..."
3,/kaggle/input/agricultural-pests-dataset/test/...,weevil,weevil,"[[[154, 148, 132], [155, 149, 132], [153, 148,..."
4,/kaggle/input/agricultural-pests-dataset/test/...,beetle,beetle,"[[[113, 88, 47], [118, 92, 51], [122, 94, 54],..."


In [ ]:
result=[]
for i in train.filename:
    result_i= model.predict_from_file(i)
    result.append(result_i.prediction)
result[:5]

['snail', 'wasp', 'bees', 'grasshopper', 'weevil']

In [ ]:
train=train.assign(prediction=result)
train.head()

,filename,label,prediction,image
0,/kaggle/input/agricultural-pests-dataset/train...,snail,snail,"[[[170, 231, 142], [231, 254, 201], [255, 254,..."
1,/kaggle/input/agricultural-pests-dataset/train...,wasp,wasp,"[[[104, 150, 42], [105, 151, 43], [105, 151, 4..."
2,/kaggle/input/agricultural-pests-dataset/train...,bees,bees,"[[[17, 20, 11], [17, 20, 11], [16, 19, 10], [1..."
3,/kaggle/input/agricultural-pests-dataset/train...,grasshopper,grasshopper,"[[[168, 169, 46], [165, 168, 46], [165, 169, 4..."
4,/kaggle/input/agricultural-pests-dataset/train...,weevil,weevil,"[[[143, 177, 65], [142, 178, 65], [141, 178, 6..."


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train['label'],train['prediction']))

              precision    recall  f1-score   support

        ants       0.98      0.97      0.97       400
        bees       0.97      0.99      0.98       405
      beetle       0.87      0.94      0.90       331
catterpillar       0.97      0.93      0.95       329
  earthworms       0.96      0.98      0.97       246
      earwig       0.97      0.94      0.95       390
 grasshopper       0.98      0.97      0.97       390
        moth       1.00      0.99      0.99       397
        slug       0.98      0.96      0.97       316
       snail       1.00      1.00      1.00       405
        wasp       0.99      0.98      0.99       392
      weevil       0.98      0.99      0.99       394

    accuracy                           0.97      4395
   macro avg       0.97      0.97      0.97      4395
weighted avg       0.97      0.97      0.97      4395



In [ ]:
print(classification_report(test['label'],test['prediction']))

              precision    recall  f1-score   support

        ants       0.91      0.98      0.94        99
        bees       0.90      0.97      0.93        95
      beetle       0.57      0.67      0.62        85
catterpillar       0.80      0.72      0.76       105
  earthworms       0.82      0.83      0.83        77
      earwig       0.85      0.74      0.79        76
 grasshopper       0.89      0.79      0.84        95
        moth       0.96      0.91      0.93       100
        slug       0.81      0.81      0.81        75
       snail       0.96      0.98      0.97        95
        wasp       0.97      0.96      0.97       106
      weevil       0.93      0.97      0.95        91

    accuracy                           0.87      1099
   macro avg       0.86      0.86      0.86      1099
weighted avg       0.87      0.87      0.87      1099



In [ ]:
!pip install lobe[cpu]
import pandas as pd
from lobe import ImageModel
from sklearn.metrics import classification_report
train = pd.read_csv('/kaggle/input/agricultural-pests-dataset/train.csv')
test = pd.read_csv('/kaggle/input/agricultural-pests-dataset/test.csv')
train['filename'] = train['filename'].str.replace('/kaggle/input/agricultural-pests-image-dataset/', '/kaggle/input/agricultural-pests-dataset/train/')
test['filename'] = test['filename'].str.replace('/kaggle/input/agricultural-pests-image-dataset/', '/kaggle/input/agricultural-pests-dataset/test/')

model = ImageModel.load('/kaggle/input/resnet-50-v2-agricultural-pests-classification/train TensorFlow')


test_results = []
for file_path in test['filename']:
    result = model.predict_from_file(file_path)
    test_results.append(result.prediction)


test['prediction'] = test_results

train_results = []
for file_path in train['filename']:
    result = model.predict_from_file(file_path)
    train_results.append(result.prediction)

# Assign predictions to the train dataframe
train['prediction'] = train_results

print("Train Data Classification Report:")
print(classification_report(train['label'], train['prediction']))

print("\nTest Data Classification Report:")
print(classification_report(test['label'], test['prediction']))

Train Data Classification Report:
              precision    recall  f1-score   support

        ants       0.98      0.97      0.97       400
        bees       0.97      0.99      0.98       405
      beetle       0.87      0.94      0.90       331
catterpillar       0.97      0.93      0.95       329
  earthworms       0.96      0.98      0.97       246
      earwig       0.97      0.94      0.95       390
 grasshopper       0.98      0.97      0.97       390
        moth       1.00      0.99      0.99       397
        slug       0.98      0.96      0.97       316
       snail       1.00      1.00      1.00       405
        wasp       0.99      0.98      0.99       392
      weevil       0.98      0.99      0.99       394

    accuracy                           0.97      4395
   macro avg       0.97      0.97      0.97      4395
weighted avg       0.97      0.97      0.97      4395


Test Data Classification Report:
              precision    recall  f1-score   support

        an

In [ ]:
import numpy as np
import tensorflow as tf
from lobe.image_utils import ensure_rgb_format
from PIL import Image

def predict_with_unknown_class(image_path, model):
    # Load the image
    image = Image.open(image_path)
    # Ensure the image is in RGB format
    image = ensure_rgb_format(image)
    # Convert the PIL image to a numpy array
    img_array = np.array(image)
    # Expand the dimensions to match the model's input shape
    img_array = np.expand_dims(img_array, axis=0)
    # Make a prediction using the model
    predictions = model.predict(img_array)
    # Get the predicted class from the model's output
    predicted_class = np.argmax(predictions)
    return predicted_class


In [ ]:
import numpy as np
import tensorflow as tf
from lobe.image_utils import ensure_rgb_format
from PIL import Image

# Assuming you have a list of class names, e.g. class_names = ["insect1", "insect2", ...]
class_names = ["insect1", "insect2", "insect3"]

# Load the saved model
model = tf.keras.models.load_model('/kaggle/input/resnet-50-v2-agricultural-pests-classification/train TensorFlow')

def predict_with_unknown_class(image_path, model, class_names, threshold=0.5):
    # Load the image
    img = Image.open(image_path)
    # Preprocess the image (resize, normalize) based on your model's requirements
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img_array = np.expand_dims(img, axis=0)
    # Make a prediction using the model
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    prediction_confidence = np.max(predictions)
    # Return the predicted class if the confidence is above the threshold
    if prediction_confidence >= threshold:
        return predicted_class
    else:
        return "unknown insect"

# Example usage
image_path = "/content/pachapuzhu.jpg"
predicted_class = predict_with_unknown_class(image_path, model, class_names)
print(f"Predicted Class: {predicted_class}")

IndexError: list index (0) out of range

In [ ]:
import numpy as np
import tensorflow as tf
from lobe.image_utils import ensure_rgb_format
from PIL import Image

# Assuming you have a list of class names, e.g. class_names = ["insect1", "insect2", ...]
class_names = ["insect1", "insect2", "insect3"]

def predict_with_unknown_class(image_path, model, class_names, threshold=0.5):
    # Load the image
    img = Image.open(image_path)
    # Preprocess the image (resize, normalize) based on your model's requirements
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img_array = np.expand_dims(img, axis=0)
    # Make a prediction using the model
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    prediction_confidence = np.max(predictions)
    # Return the predicted class if the confidence is above the threshold
    if prediction_confidence >= threshold:
        return predicted_class
    else:
        return "unknown insect"

# Example usage
image_path = "/content/pachapuzhu.jpg"
predicted_class = predict_with_unknown_class(image_path, model, class_names)
print(f"Predicted Class: {predicted_class}")

AttributeError: 'numpy.ndarray' object has no attribute 'convert'

In [ ]:
import firebase_admin
from firebase_admin import credentials , storage

In [ ]:
cred = credentials.Certificate('/content/bit-896e2-firebase-adminsdk-9hgyp-c2ae216620.json')
firebase_app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://bit-896e2.appspot.com'
})

In [ ]:
import firebase_admin
from firebase_admin import storage

# ... (your Firebase app initialization code)

storage_client = storage.bucket()  # Use get_bucket() directly

def download_image_from_firebase(image_id):
  blob = storage_client.blob(f"pachapuzhu/{image_id}.jpg")
  blob.download_to_filename(f"downloaded_images/{image_id}.jpg")
